# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [24]:
# import libraries
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge 
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import root_mean_squared_error, mean_absolute_error

In [25]:
import warnings
warnings.filterwarnings('ignore')

In [26]:
# CONSTANTS
RND_SEED = 42
CV_FOLDS = 5
DATA_DIR = '../../05_src/data/fires/'
SCORING = 'neg_root_mean_squared_error' #,'neg_mean_absolute_error']

In [27]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [28]:
targ_col = 'area'
X = fires_dt.drop(columns = [targ_col]) # features
y = fires_dt[targ_col]

# split into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RND_SEED)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


In [29]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()

### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [30]:
# Preproc 1: scaling and one-hot encoder
preproc1 = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='if_binary'), cat_cols)
    ], remainder = 'drop'
)

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [31]:
# Preproc 2: scaling + nonlinear transform and one-hot encoder
pipe_num_yj = Pipeline(
    steps = [
        ('transformer', PowerTransformer(method = 'yeo-johnson')),
        ('standardizer', StandardScaler())
])
pipe_cat = OneHotEncoder(handle_unknown='ignore',drop='if_binary')

preproc2 = ColumnTransformer(
    transformers = [
        ('num', pipe_num_yj, num_cols),
        ('cat', pipe_cat, cat_cols)
    ], remainder = 'drop'
)

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [32]:
# Pipeline A = preproc1 + baseline
pipe_a = Pipeline(
    steps = [
        ('preprocessing', preproc1),
        ('regressor', Ridge(random_state = RND_SEED))
    ]
)

# # fit and evaluate using cross-validation
# cv_pipe_a = pd.DataFrame(cross_validate(pipe_a, X_train, y_train, cv=CV_FOLDS, scoring=SCORING))
# rmse_pipe_a = -cv_pipe_a.mean().test_neg_root_mean_squared_error.tolist()
# fit_a = pipe_a.fit(X_train, y_train)
# score_a = pipe_a.score(X_test, y_test)

In [33]:
# Pipeline B = preproc2 + baseline
pipe_b= Pipeline(
    steps = [
        ('preprocessing', preproc2),
        ('regressor', Ridge(random_state = RND_SEED))
    ]
)

In [34]:
# Pipeline C = preproc1 + advanced model
pipe_c = Pipeline(
    steps = [
        ('preprocessing', preproc1),
        ('regressor', RandomForestRegressor(random_state = RND_SEED))
    ]
)

In [35]:
# Pipeline D = preproc2 + advanced model
pipe_d = Pipeline(
    steps = [
        ('preprocessing', preproc2),
        ('regressor', RandomForestRegressor(random_state = RND_SEED))
    ]
)
    

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [36]:
# for pipe_a and pipe_b using ridge regression
param_grid_lr = {'regressor__alpha': [0.1, 1.0, 10.0, 100.0]}

# for pipe_c and pipe_d using random forest regressor
param_grid_rf = {'regressor__n_estimators': [10, 100, 250, 500]}

In [37]:
# Pipe A: Preproc1 + Ridge 
grid_cv_a = GridSearchCV(
    estimator = pipe_a,
    param_grid = param_grid_lr,
    cv = CV_FOLDS,
    scoring = SCORING
)
grid_cv_a.fit(X_train, y_train)

print(f'Pipe A: Best hyperparameters = {grid_cv_a.best_params_}')
print(f'Pipe A: Best score = {grid_cv_a.best_score_}')

Pipe A: Best hyperparameters = {'regressor__alpha': 100.0}
Pipe A: Best score = -39.42531880594738


In [38]:
# Pipe B: Preproc2 + Ridge
grid_cv_b = GridSearchCV(
    estimator = pipe_b,
    param_grid = param_grid_lr,
    cv = CV_FOLDS,
    scoring = SCORING
)
grid_cv_b.fit(X_train, y_train)

print(f'Pipe B: Best hyperparameters = {grid_cv_b.best_params_}')
print(f'Pipe B: Best score = {grid_cv_b.best_score_}')

Pipe B: Best hyperparameters = {'regressor__alpha': 100.0}
Pipe B: Best score = -38.91854213194067


In [39]:
# Pipe C: Preproc1 + Random Forest
grid_cv_c = GridSearchCV(
    estimator = pipe_c,
    param_grid = param_grid_rf,
    cv = CV_FOLDS,
    scoring = SCORING
)
grid_cv_c.fit(X_train, y_train)

print(f'Pipe C: Best hyperparameters = {grid_cv_c.best_params_}')
print(f'Pipe C: Best score = {grid_cv_c.best_score_}')

Pipe C: Best hyperparameters = {'regressor__n_estimators': 100}
Pipe C: Best score = -48.13441427676149


In [40]:
# Pipe D: Preproc2 + Random Forest
grid_cv_d = GridSearchCV(
    estimator = pipe_d,
    param_grid = param_grid_rf,
    cv = CV_FOLDS,
    scoring = SCORING
)
grid_cv_d.fit(X_train, y_train)

print(f'Pipe D: Best hyperparameters = {grid_cv_d.best_params_}')
print(f'Pipe D: Best score = {grid_cv_d.best_score_}')

Pipe D: Best hyperparameters = {'regressor__n_estimators': 100}
Pipe D: Best score = -47.4717995929726


# Evaluate

+ Which model has the best performance?

In [41]:
test_scores = {
    'Pipe A': grid_cv_a.best_estimator_.score(X_test, y_test),
    'Pipe B': grid_cv_b.best_estimator_.score(X_test, y_test),
    'Pipe C': grid_cv_c.best_estimator_.score(X_test, y_test),
    'Pipe D': grid_cv_d.best_estimator_.score(X_test, y_test)
}

for pipe, score in test_scores.items():
    print(f'{pipe}: test score = {score:.4f}')

Pipe A: test score = 0.0040
Pipe B: test score = 0.0055
Pipe C: test score = -0.0205
Pipe D: test score = -0.0195


In [42]:
best_pipe = max(test_scores, key = test_scores.get)
print(f'Best pipeline = {best_pipe}; Test score = {test_scores[best_pipe]:.4f}')

Best pipeline = Pipe B; Test score = 0.0055


Is it terrible performance? Yep. Regular linear regression probably would have been better. My goal for this assignment is demonstrating the pipeline process. 

# Export

+ Save the best performing model to a pickle file.

In [43]:
best_model = grid_cv_b.best_estimator_

In [44]:
import pickle
with open('best_performing_model.pkl','wb') as f:
    pickle.dump(best_model, f)

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [45]:
import shap # can't get shap to work

ModuleNotFoundError: No module named 'shap'

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.